---


**Importando as bibliotecas**

---



In [284]:
import numpy as np
import pandas as pd

# **1. Coletar dados**

---

**Coleta dos dados**: acidentes nas rodovias federais nordestinas em 2018

---



In [285]:
#Lê o dataset de acidentes no Brasil armazenado no arquivo csv
df = pd.read_csv("datatran2018.csv", encoding='latin', sep=";", error_bad_lines=False)

#Cria um filtro para selecionar apenas acidentes no Nordeste
filter_northeast = ((df["uf"] == "BA") | (df["uf"] == "PE") | (df["uf"] == "AL") |
                    (df["uf"] == "MA") | (df["uf"] == "CE") | (df["uf"] == "PB") |
                    (df["uf"] == "PI") | (df["uf"] == "RN") | (df["uf"] == "CE"))

#Atualiza o dataframe apenas com os acidentes no Nordeste
df = df[filter_northeast]

#Mostra o número de linhas (acidentes) e colunas (informações) do dataframe de acidentes
df.shape

(14285, 30)

# **2. Realizar pré-processamento dos dados**

---

**Definição de tipos**

---



In [286]:
#Apresenta os cinco primeiros dados do dataframe
df.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
11,99991.0,2018-01-01,segunda-feira,02:00:00,PB,230.0,4,CABEDELO,Velocidade Incompatível,Colisão traseira,Com Vítimas Feridas,Plena Noite,Decrescente,Céu Claro,Dupla,Reta,Sim,6,0,1,2,3,0,3,2,-7.006437,-34.829589,SR-PB,DEL14/1,UOP01/PB
15,99995.0,2018-01-01,segunda-feira,01:40:00,RN,101.0,"110,8",PARNAMIRIM,Animais na Pista,Atropelamento de Animal,Com Vítimas Feridas,Plena Noite,Crescente,Céu Claro,Dupla,Reta,Não,1,0,1,0,0,0,1,1,-5.953546,-35.264860,SR-RN,DEL15/1,UOP01/RN
26,100006.0,2018-01-01,segunda-feira,03:20:00,PE,232.0,58,POMBOS,Condutor Dormindo,Colisão com objeto estático,Com Vítimas Feridas,Plena Noite,Decrescente,Céu Claro,Dupla,Ponte,Sim,1,0,0,1,0,0,1,1,-8.147220,-35.397579,SR-PE,DEL11/1,UOP04/PE
28,100009.0,2018-01-01,segunda-feira,03:40:00,AL,104.0,44,BRANQUINHA,Não guardar distância de segurança,Colisão traseira,Sem Vítimas,Plena Noite,Crescente,Céu Claro,Simples,Curva,Não,2,0,0,0,1,1,0,2,-9.245987,-36.015575,SR-AL,DEL13/1,UOP02/AL
30,100011.0,2018-01-01,segunda-feira,03:30:00,PE,101.0,69,RECIFE,Condutor Dormindo,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Céu Claro,Múltipla,Reta,Sim,1,0,1,0,0,0,1,1,-8.063598,-34.944449,SR-PE,DEL11/1,UOP01/PE


*   **id**: categórico ordinal
*   **data_inversa**: categórico ordinal
*   **dia_semana**: categórico nominal
*   **horário**: categórico ordinal
*   **uf**: categórico nominal
*   **br**: categórico nominal
*   **km**: contínuo
*   **município**: categórico nominal
*   **causa_acidente**: categórico nominal
*   **tipo_acidente**: categórico nominal
*   **classificacao_acidente**: categórico nominal
*   **fase_dia**: categórico nominal
*   **sentido_via**: categórico nominal
*   **condicao_metereologica**: categórico nominal
*   **tipo_pista**: categórico nominal
*   **tracado_via**: categórico nominal
*   **uso_solo**: categórico binário
*   **pessoas**: contínuo
*   **mortos**: contínuo
*   **feridos_leves**: contínuo
*   **feridos_graves**: contínuo
*   **ilesos**: contínuo
*   **ignorados**: contínuo
*   **feridos**: contínuo
*   **veiculos**: contínuo
*   **latitude**: contínuo
*   **longitude**: contínuo
*   **regional**: categórico nominal
*   **delegacia**: categórico nominal
*   **uop**: categórico nominal

**Categóricos ordinais**: 3

**Categóricos binários**: 1

**Categóricos nominais**: 15

**Contínuos**: 11

---

**Tratamento de dados ausentes**: como apresentado abaixo, 28 amostras não possuem os dados relacionados à BR e ao KM onde ocorreu o acidente. Devido a impossibilidade de inputar estes dados, as amostras foram removidas.

---



In [287]:
#Verifica a quantidade de NaN em cada coluna do dataset
df.isna().sum()

id                         0
data_inversa               0
dia_semana                 0
horario                    0
uf                         0
br                        28
km                        28
municipio                  0
causa_acidente             0
tipo_acidente              0
classificacao_acidente     0
fase_dia                   0
sentido_via                0
condicao_metereologica     0
tipo_pista                 0
tracado_via                0
uso_solo                   0
pessoas                    0
mortos                     0
feridos_leves              0
feridos_graves             0
ilesos                     0
ignorados                  0
feridos                    0
veiculos                   0
latitude                   0
longitude                  0
regional                   0
delegacia                  0
uop                        2
dtype: int64

In [288]:
#Remove as linhas que possuem NaN em alguma coluna
df.dropna(inplace = True)
#Informa as dimensões do dataset
df.shape

(14257, 30)

---

**Normalização e Discretização**: O processo de normalização consistiu na transformação dos dados 'causa_acidente' e 'tipo_acidente' em níveis de 0 a 1 que indicam do melhor ao pior caso. Em relação ao processo de discretização, foram definidos intervalos que agrupam os acidentes por quantidade pessoas envolvidas a partir do processamento do dado 'pessoas'. Ambos os processos resultaram em novas colunas no dataframe.

---

In [289]:
#Lista todas as causas de acidentes
df["causa_acidente"].value_counts()

Falta de Atenção à Condução                                                       4962
Desobediência às normas de trânsito pelo condutor                                 1653
Ingestão de Álcool                                                                1161
Não guardar distância de segurança                                                1016
Velocidade Incompatível                                                            904
Defeito Mecânico no Veículo                                                        706
Animais na Pista                                                                   599
Falta de Atenção do Pedestre                                                       562
Condutor Dormindo                                                                  458
Pista Escorregadia                                                                 452
Ultrapassagem Indevida                                                             389
Defeito na Via                             

In [290]:
#Tupla de causas de acidente onde não existe a culpa do condutor
no_driver_irresponsibility = ("Animais na Pista", "Falta de Atenção do Pedestre", "Pista Escorregadia", "Defeito na Via", "Mal Súbito", "Sinalização da via insuficiente ou inadequada ", "Agressão Externa", "Desobediência às normas de trânsito pelo pedestre")

#Tupla de causas de acidente onde existe baixa culpa do condutor
low_driver_irresponsibility = ("Restrição de Visibilidade", "Deficiência ou não Acionamento do Sistema de Iluminação/Sinalização do Veículo", "Fenômenos da Natureza")

#Tupla de causas de acidente onde existe culpa moderada do condutor
moderate_driver_irresponsibility = ("Avarias e/ou desgaste excessivo no pneu", "Objeto estático sobre o leito carroçável", "Carga excessiva e/ou mal acondicionada")

#Tupla de causas de acidente onde existe alta culpa do condutor
high_driver_irresponsibility = ("Não guardar distância de segurança", "Defeito Mecânico no Veículo", "Condutor Dormindo")

#Tupla de causas de acidente onde existe culpa muito alta do condutor
very_high_driver_irresponsibility = ("Falta de Atenção à Condução", "Desobediência às normas de trânsito pelo condutor", "Ingestão de Álcool", "Velocidade Incompatível", "Ultrapassagem Indevida", "Ingestão de álcool e/ou substâncias psicoativas pelo pedestre", "Ingestão de Substâncias Psicoativas")

#Transforma a causa do acidente em um índice que representa o nível de culpa do condutor
def normalize_accident_cause(accident_cause):
  #Variável para o nível de culpa do condutor
  level = None

  #Níveis de culpa
  if accident_cause in no_driver_irresponsibility:
    level = 0.0
  elif accident_cause in low_driver_irresponsibility:
    level = 0.25
  elif accident_cause in moderate_driver_irresponsibility:
    level = 0.5
  elif accident_cause in high_driver_irresponsibility:
    level = 0.75
  elif accident_cause in very_high_driver_irresponsibility:
    level = 1.0

  #Retorna o nível de culpa do condutor
  return level

#Cria uma nova coluna com dado categórico ordinal a partir do cálculo
#do nível de culpa do condutor no acidente
df["causa_acidente_normalizado"] = df["causa_acidente"].apply(normalize_accident_cause)

#Mostra a coluna primitiva e a derivada a ela
df[["causa_acidente", "causa_acidente_normalizado"]].head()

,causa_acidente,causa_acidente_normalizado
11,Velocidade Incompatível,1.00
15,Animais na Pista,0.00
26,Condutor Dormindo,0.75
28,Não guardar distância de segurança,0.75
30,Condutor Dormindo,0.75


In [291]:
#Lista todas os tipos de acidentes
df["tipo_acidente"].value_counts()

Colisão traseira                   2623
Colisão transversal                1914
Saída de leito carroçável          1875
Colisão lateral                    1772
Colisão frontal                    1292
Tombamento                          944
Atropelamento de Pedestre           878
Colisão com objeto estático         694
Capotamento                         663
Queda de ocupante de veículo        543
Atropelamento de Animal             478
Engavetamento                       251
Incêndio                            168
Colisão com objeto em movimento      77
Danos eventuais                      56
Derramamento de carga                29
Name: tipo_acidente, dtype: int64

In [292]:
#Tupla de acidentes de risco mínimo
very_low_risk = ("Colisão traseira", "Danos eventuais", "Derramamento de carga")

#Tupla de acidentes de baixo risco
low_risk = ("Queda de ocupante de veículo", "Atropelamento de Animal", "Colisão com objeto em movimento ")

#Tupla de acidentes de risco moderado
moderate_risk = ("Colisão com objeto estático", "Engavetamento")

#Tupla de acidentes de alto risco
high_risk = ("Colisão transversal", "Tombamento", "Saída de leito carroçável", "Incêndio")

#Tupla de acidentes de risco máximo
very_high_risk = ("Colisão lateral", "Colisão frontal", "Atropelamento de Pedestre", "Capotamento")

#Transforma o tipo do acidente em um índice que representa o nível de risco
def normalize_accident_type(accident_type):
  #Variável para o nível de risco do acidente
  level = None

  #Níveis de risco
  if accident_type in very_low_risk:
    level = 0.0
  elif accident_type in low_risk:
    level = 0.25
  elif accident_type in moderate_risk:
    level = 0.5
  elif accident_type in high_risk:
    level = 0.75
  elif accident_type in very_high_risk:
    level = 1.0

  #Retorna o nível de risco
  return level

#Cria uma nova coluna com dado categórico ordinal a partir do cálculo
#do nível de risco de vítimas no acidente
df["tipo_acidente_normalizado"] = df["tipo_acidente"].apply(normalize_accident_type)

#Mostra a coluna primitiva e a derivada a ela
df[["tipo_acidente", "tipo_acidente_normalizado"]].head()

,tipo_acidente,tipo_acidente_normalizado
11,Colisão traseira,0.00
15,Atropelamento de Animal,0.25
26,Colisão com objeto estático,0.50
28,Colisão traseira,0.00
30,Saída de leito carroçável,0.75


In [293]:
#Acidente com o maior número de pessoas envolvidas
df["pessoas"].max()

55

In [294]:
#Discretiza o número de pessoas envolvidas nos acidentes em intervalos de 10
def discretize_n_people(people):
  #Variável para o intervalo
  range = None

  #Intervalos considerados a partir do máximo de pessoas envolvidas no dataset
  if people >= 0 and people < 10:
    range = "[0, 10)"
  elif people >= 10 and people < 20:
    range = "[10, 20)"
  elif people >= 20 and people < 30:
    range = "[20, 30)"
  elif people >= 30 and people < 40:
    range = "[30, 40)"
  elif people >= 40 and people < 50:
    range = "[40, 50)"
  elif people >= 50 and people <= 60:
    range = "[50, 60]"

  #Retorna o intervalo calulado para determinado número de pessoas
  return range

#Cria uma nova coluna com dado categórico nominal a partir do cálculo do 
#intervalo de pessoas para cada linha
df["pessoas_discretizadas"] = df["pessoas"].apply(discretize_n_people)

#Mostra a coluna primitiva e a derivada a ela
df[["pessoas", "pessoas_discretizadas"]].head()

,pessoas,pessoas_discretizadas
11,6,"[0, 10)"
15,1,"[0, 10)"
26,1,"[0, 10)"
28,2,"[0, 10)"
30,1,"[0, 10)"


---

**Limpeza de dados**: diante da ausência de outliers, incorretos e dados duplicados, a limpeza foi realizada apenas para dados irrelevantes para esta análise: id, km, municipio, sentido_via, feridos, regional, delegacia e uop

---

In [302]:
#Remove as colunas irrelevantes
df.drop(["id", "km", "municipio", "sentido_via", "feridos", "regional", "delegacia", "uop"], axis = 1, inplace = True)
#Mostra o novo dataframe
df.head()

,data_inversa,dia_semana,horario,uf,br,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,veiculos,latitude,longitude,causa_acidente_normalizado,tipo_acidente_normalizado,pessoas_discretizadas
11,2018-01-01,segunda-feira,02:00:00,PB,230.0,Velocidade Incompatível,Colisão traseira,Com Vítimas Feridas,Plena Noite,Céu Claro,Dupla,Reta,Sim,6,0,1,2,3,0,2,-7.006437,-34.829589,1.00,0.00,"[0, 10)"
15,2018-01-01,segunda-feira,01:40:00,RN,101.0,Animais na Pista,Atropelamento de Animal,Com Vítimas Feridas,Plena Noite,Céu Claro,Dupla,Reta,Não,1,0,1,0,0,0,1,-5.953546,-35.264860,0.00,0.25,"[0, 10)"
26,2018-01-01,segunda-feira,03:20:00,PE,232.0,Condutor Dormindo,Colisão com objeto estático,Com Vítimas Feridas,Plena Noite,Céu Claro,Dupla,Ponte,Sim,1,0,0,1,0,0,1,-8.147220,-35.397579,0.75,0.50,"[0, 10)"
28,2018-01-01,segunda-feira,03:40:00,AL,104.0,Não guardar distância de segurança,Colisão traseira,Sem Vítimas,Plena Noite,Céu Claro,Simples,Curva,Não,2,0,0,0,1,1,2,-9.245987,-36.015575,0.75,0.00,"[0, 10)"
30,2018-01-01,segunda-feira,03:30:00,PE,101.0,Condutor Dormindo,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Céu Claro,Múltipla,Reta,Sim,1,0,1,0,0,0,1,-8.063598,-34.944449,0.75,0.75,"[0, 10)"


#**3. Apresentar estatísticas descritivas dos dados com visualizações**